In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


from input_generator import load_dataset, load_dataset_with_lables
from data_parser import get_tags_and_labels, get_vocab, get_tags

d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
# Get dataframe with tags and category_id columns
csvpath = 'data/CAvideos.cvs'
tab_dataframe = get_tags_and_labels(csvpath)

# Creating different coloumn based on different tags
new = tab_dataframe["tags"].str.split(".", expand = True)

max_of_tags = 32

for i in range(0, max_of_tags):
    name = "tags"+str(i)
    tab_dataframe[name] = new[i]

# Dropping old tags columns 
tab_dataframe.drop(columns =["tags"], inplace = True) 
tab_dataframe.fillna("NA", inplace = True) 


tab_dataframe

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,...,tags22,tags23,tags24,tags25,tags26,tags27,tags28,tags29,tags30,tags31
0,10,Eminem,Walk,On,Water,Aftermath/Shady/Interscope,Rap,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,23,plush,bad unboxing,unboxing,fan mail,idubbbztv,idubbbztv2,things,best,packages,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,23,racist superman,rudy,mancuso,king,bach,racist,superman,love,rudy mancuso poo bear black white official mus...,...,Lele Pons,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,24,ryan,higa,higatv,nigahiga,i dare you,idy,rhpc,dares,no truth,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,10,edsheeran,ed sheeran,acoustic,live,cover,official,remix,official video,lyrics,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,25,#DramaAlert,Drama,Alert,DramaAlert,keemstar,youtube news,jake paul,team 10,alissa violet,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,23,Funny Moments,Montage video games,gaming,Vanoss,VanossGaming,Gameplay,Secret,Puncake,Parody,...,vanoss gaming school,doc,documentary,NA,NA,NA,NA,NA,NA,NA
7,22,SHANtell martin,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8,24,logan paul vlog,logan paul,logan,paul,olympics,logan paul youtube,vlog,daily,comedy,...,logan paul arab,arabic,arab,logan paul age,the maverick,maverick merch,NA,NA,NA,NA
9,22,God,Sheldon Cooper,Young Sheldon,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [3]:
# Getting leaning vocab
vocab = get_vocab(get_tags(csvpath))
voc_di = {vo:i for i,vo in enumerate(vocab)}
voc_di['NA'] = 0

In [4]:
# Mapping tags to indexes in vocab
for index in range(0, max_of_tags):
    tab_dataframe['tags{}'.format(index)] = tab_dataframe['tags{}'.format(index)].map(voc_di)
    
tab_dataframe

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,...,tags22,tags23,tags24,tags25,tags26,tags27,tags28,tags29,tags30,tags31
0,10,65883.0,145851.0,82925.0,88240.0,97431.0,149147.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23,58399.0,57580.0,29621.0,117549.0,133976.0,95133.0,48365.0,2677.0,3646.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,23,105365.0,82779.0,19442.0,91339.0,66636.0,71121.0,81052.0,70632.0,17189.0,...,7839.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,24,133492.0,7013.0,131530.0,121963.0,78043.0,28109.0,146851.0,138451.0,60743.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,74642.0,56831.0,36115.0,43386.0,120685.0,135238.0,102579.0,183.0,38316.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,25,112011.0,110391.0,67947.0,115853.0,5380.0,121580.0,141352.0,99830.0,63312.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,23,61797.0,99548.0,46695.0,26719.0,147911.0,66095.0,126749.0,37725.0,21149.0,...,117459.0,72065.0,76233.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,22,15656.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,24,3758.0,144686.0,15740.0,58773.0,131969.0,13548.0,19075.0,150739.0,89820.0,...,26797.0,32112.0,72624.0,81979.0,24828.0,105156.0,0.0,0.0,0.0,0.0
9,22,150831.0,4291.0,55190.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
for column in tab_dataframe.columns:
    if column == 'category_id':
        pass
    else:
        if tab_dataframe[column].max() == 0:
            tab_dataframe[column] = tab_dataframe[column]

        else:
            tab_dataframe[column] = tab_dataframe[column] / tab_dataframe[column].max()

tab_dataframe

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,...,tags22,tags23,tags24,tags25,tags26,tags27,tags28,tags29,tags30,tags31
0,10,0.434857,0.962745,0.547439,0.582404,0.643100,0.984417,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,23,0.385459,0.380079,0.195547,0.775850,0.884318,0.627907,0.319249,0.017671,0.024066,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,23,0.695456,0.546414,0.128349,0.602858,0.439836,0.469421,0.535011,0.466246,0.113461,...,0.051758,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,24,0.881106,0.046292,0.868311,0.804983,0.515129,0.185528,0.969339,0.913922,0.400952,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,10,0.492670,0.375134,0.238417,0.286357,0.796590,0.892613,0.677107,0.001208,0.252916,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,25,0.739322,0.728678,0.448560,0.764656,0.035511,0.802466,0.933041,0.658983,0.417909,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,23,0.407888,0.657104,0.308263,0.176351,0.976297,0.436248,0.836649,0.249025,0.139600,...,0.775542,0.475702,0.503218,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,22,0.103337,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,24,0.024804,0.955055,0.103909,0.387915,0.871071,0.089421,0.125911,0.995036,0.592883,...,0.176932,0.211972,0.479395,0.541205,0.163948,0.694259,0.000000,0.000000,0.000000,0.000000
9,22,0.995551,0.028324,0.364343,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
# Create train,validation and test examples from dataframe
# train_ds, val_ds, test_ds = load_dataset(tab_dataframe)
train_ds, test_ds, train_lb, test_lb = load_dataset_with_lables(tab_dataframe, 'category_id')


In [10]:
print(train_ds)

Int64Index([19880, 15570, 19419, 32071, 40571, 36811, 35693, 16009,  2809,
            30680,
            ...
            35731, 22895,  6448, 32362, 27443, 16760, 29841, 12596,  3837,
            33207],
           dtype='int64', length=32704)


In [12]:
model = tf.keras.Sequential([
    layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.01), input_shape = (32, 25)),
    layers.Dropout(0.5),
    layers.Dense(128, kernel_regularizer=keras.regularizers.l2(0.01), input_shape = (32, 64)),
    layers.Dense(64, activation='sigmoid', input_shape = (32, 128)),
    layers.Dense(44, activation='softmax', kernel_regularizer=keras.regularizers.l2(0.01), input_shape = (32, 64))
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', 'binary_crossentropy'],
              )

history = model.fit(train_ds, train_lb,
          batch_size=32,
          epochs=5)


ValueError: Error when checking input: expected dense_input to have 3 dimensions, but got array with shape (32704, 1)